In [1]:
%load_ext autoreload
%autoreload 2

In [22]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import OrdinalEncoder
from sklearn.impute import KNNImputer

In [ ]:
df = pd.read_csv('Data/pf_suvs_i302_1s2025.csv')
df = df.drop(df.columns[0], axis=1)
mask = df['Moneda'] == '$'
df.loc[mask, 'Precio'] = df.loc[mask, 'Precio'] / 1185
df.drop(columns=['Moneda'], inplace=True)
df.drop(columns = ['Tipo de carrocería'],inplace=True)
idx = df[df['Año'] == 436694.0].index
df.drop(idx, inplace=True)

df['Puertas'] = df['Puertas'].replace(60252.0, np.nan)
enc = OrdinalEncoder()
df['Modelo_code'] = enc.fit_transform(df[['Modelo']])
imputer = KNNImputer(n_neighbors=5)
cols = ['Modelo_code', 'Año', 'Puertas']
df[cols] = imputer.fit_transform(df[cols])
df['Puertas'] = np.round(df['Puertas']).astype(int)
df.drop('Modelo_code', axis=1, inplace=True)
print(df['Puertas'].value_counts(dropna=False))

In [ ]:
print(df.columns)
anomalous_models = df.loc[df['Puertas'] == 60252.0, 'Modelo'].unique()
anomalous_years  = df.loc[df['Puertas'] == 60252.0, 'Año'].unique()
mask = (
    df['Año'].isin(anomalous_years) &
    df['Modelo'].isin(anomalous_models)
)

df_filtered = df.loc[mask, ['Modelo','Puertas','Año']]

result = (
    df_filtered
      .drop_duplicates()
      .sort_values(['Año','Modelo'])
)

print(result)

Index(['Marca', 'Modelo', 'Año', 'Versión', 'Color', 'Tipo de combustible',
       'Puertas', 'Transmisión', 'Motor', 'Con cámara de retroceso',
       'Kilómetros', 'Título', 'Precio', 'Descripción', 'Tipo de vendedor'],
      dtype='object')
      Modelo  Puertas     Año
889     RAV4      5.0  2009.0
7884    RAV4  60252.0  2009.0
3395    RAV4      5.0  2019.0
11110   RAV4      4.0  2019.0
68        X1      5.0  2019.0
1602      X1  60252.0  2019.0
445     RAV4      5.0  2022.0
176     Taos      5.0  2022.0
7633    Taos  60252.0  2022.0
1900      X1      5.0  2022.0
Puertas
5    17734
4      399
3      105
2       10
6        3
7        2
Name: count, dtype: int64


Modelo
Ecosport       1569
Tracker        1511
2008           1144
Duster         1048
Compass         916
               ... 
Nativa            2
Defender          2
208               2
C4 Aircross       2
F-PACE            2
Name: count, Length: 137, dtype: int64
